In [1]:
#https://www.kaggle.com/datasets/muratkokludataset/pistachio-image-dataset

In [1]:
#data directory
data_dir = 'C:\\projects\\all-projects\\vislearn\\pista\\Pistachio_Image_Dataset\\Pistachio_Image_Dataset'
#show folder names
import os
os.listdir(data_dir)


['Kirmizi_Pistachio', 'Pistachio_Image_Dataset_Request.txt', 'Siirt_Pistachio']

In [2]:
#kirmizi directory
kirmizi_dir = "C:\\projects\\all-projects\\vislearn\\pista\\Pistachio_Image_Dataset\\Pistachio_Image_Dataset\\Kirmizi_Pistachio"
#show all files in kirmizi directory
kirmizi_files = os.listdir(kirmizi_dir)
print(kirmizi_files[:10])

['kirmizi (1).jpg', 'kirmizi (10).jpg', 'kirmizi (11).jpg', 'kirmizi (12).jpg', 'kirmizi (13).jpg', 'kirmizi (14).jpg', 'kirmizi (15).jpg', 'kirmizi (16).jpg', 'kirmizi (17).jpg', 'kirmizi (18).jpg']


In [3]:
#siirt directory
siirt_dir = "C:\\projects\\all-projects\\vislearn\\pista\\Pistachio_Image_Dataset\\Pistachio_Image_Dataset\\Siirt_Pistachio"
#show all files in siirt directory
siirt_files = os.listdir(siirt_dir)
print(siirt_files[:10])

['siirt (1).jpg', 'siirt (10).jpg', 'siirt (11).jpg', 'siirt (12).jpg', 'siirt (13).jpg', 'siirt (14).jpg', 'siirt (15).jpg', 'siirt (16).jpg', 'siirt (17).jpg', 'siirt (18).jpg']


In [4]:
#print number of images in each directory
print('total training kirmizi images:', len(os.listdir(kirmizi_dir)))
print('total training siirt images:', len(os.listdir(siirt_dir)))


total training kirmizi images: 1232
total training siirt images: 916


In [10]:
#all of the images
import glob
images = glob.glob("C:\\projects\\all-projects\\vislearn\\pista\\Pistachio_Image_Dataset\\Pistachio_Image_Dataset\\*\\*.jpg")
#number of images
import pandas as pd
print("Number of images: ", len(images))

images = pd.Series(images)

ImportError: this version of pandas is incompatible with numpy < 1.20.3
your numpy version is 1.19.5.
Please upgrade numpy to >= 1.20.3 to use this pandas version